Import the libraries, as always, and read in the data

In [1]:
from galyleo.galyleo_table import GalyleoTable
from galyleo.galyleo_constants import GALYLEO_STRING, GALYLEO_NUMBER
from galyleo.galyleo_jupyterlab_client import GalyleoClient
import csv

def state_code(current_code):
    if current_code == '': return ''
    canada_codes = {'ab', 'bc', 'sk', 'mn', 'on', 'qc', 'pe', 'nb', 'ns', 'nl', 'nt', 'nu', 'yt'}
    country = 'CA' if current_code in canada_codes else 'US'
    return f'{country}-{current_code.upper()}'

def cleanse_row(row):
    values = [entry.strip() for entry in row]
    return  [int(values[i]) for i in range(3) ] + values[3:5] + [state_code(values[5]), values[6], float(values[7])]

# read the file and make a table
with open('ufos.csv', 'r') as ufo_file:
    reader = csv.reader(ufo_file)
    column_names = next(reader)
    data = [cleanse_row(row) for row in reader]
ufo_file.close()

In [2]:
data[:10]

[[588, 10, 1949, 'night', 'us', 'US-TX', 'cylinder', 2700.0],
 [588, 10, 1949, 'night', 'us', 'US-TX', 'light', 7200.0],
 [2779, 10, 1955, 'afternoon', 'gb', '', 'circle', 20.0],
 [3145, 10, 1956, 'night', 'us', 'US-TX', 'circle', 20.0],
 [4606, 10, 1960, 'night', 'us', 'US-HI', 'light', 900.0],
 [4971, 10, 1961, 'night', 'us', 'US-TN', 'sphere', 300.0],
 [6432, 10, 1965, 'night', 'gb', '', 'circle', 180.0],
 [6432, 10, 1965, 'night', 'us', 'US-CT', 'disk', 1200.0],
 [6797, 10, 1966, 'night', 'us', 'US-AL', 'disk', 180.0],
 [6797, 10, 1966, 'night', 'us', 'US-FL', 'disk', 120.0]]

In [3]:
column_names = [name.strip() for name in column_names]
schema = [(column_names[i], GALYLEO_NUMBER) for i in range(3)] + [(column_names[i], GALYLEO_STRING) for i in range(3,7)] + [(column_names[7], GALYLEO_STRING)]
table = GalyleoTable('ufos')
table.load_from_schema_and_data(schema, data)

Aggregate by year, month, country

In [4]:
sightings_by_country_year_month = table.aggregate_by([ 'country', 'year', 'month'], 'count', 'aggregate_cym')

Create a Dashboard using the Launcher or the File menu, then execute the next cell to send the data to it

In [5]:
client = GalyleoClient()
client.send_data_to_dashboard(sightings_by_country_year_month)

Aggregate by year, and country 

In [7]:
sightings_by_country_year = table.aggregate_by( [ 'country', 'year', ], 'count', 'aggregate_cy')
client.send_data_to_dashboard(sightings_by_country_year)

Use this to get North American sightings

In [8]:
north_american_sightings = table.filter_by_function("country", lambda x: x in {'ca', 'us'}, 'north_america_table')

Aggregate by state, year, month and by state and month

In [9]:
sightings_by_state_year_month = north_american_sightings.aggregate_by([ 'state', 'year', 'month'], 'count', 'aggregate_sym')
sightings_by_state_year = north_american_sightings.aggregate_by([ 'state', 'year'], 'count', 'aggregate_sy')
client.send_data_to_dashboard(sightings_by_state_year)
client.send_data_to_dashboard(sightings_by_state_year_month)

Aggregate by state, year, month, type and by country, year, month, type

In [10]:
sightings_by_state_year_type = north_american_sightings.aggregate_by([ 'state', 'year', 'type'], 'count', 'aggregate_syt')
sightings_by_country_year_type = table.aggregate_by([ 'country', 'year', 'type'], 'count', 'aggregate_cyt')
client.send_data_to_dashboard(sightings_by_state_year_type)
client.send_data_to_dashboard(sightings_by_country_year_type)